In [100]:
import pandas as pd

refs_df = pd.read_csv(
    "../data/processed/refs.csv",
    usecols=lambda c: c != "context"
)

refs_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265726 entries, 0 to 265725
Data columns (total 17 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   paper_id              265726 non-null  int64  
 1   raw                   265644 non-null  object 
 2   co_cited_count        265726 non-null  int64  
 3   section_id            259027 non-null  object 
 4   paragraph_id          265684 non-null  object 
 5   sentence_id           265726 non-null  object 
 6   sentence_seq_number   265726 non-null  int64  
 7   reference_seq_number  265726 non-null  int64  
 8   author                227628 non-null  object 
 9   page                  74403 non-null   float64
 10  year                  222883 non-null  float64
 11  title                 265726 non-null  object 
 12  filename              265726 non-null  object 
 13  sentence_count        265726 non-null  int64  
 14  reference_count       265726 non-null  int64  
 15  

In [101]:
import pandas as pd

# 1. Lê o CSV com o mapeamento autor → tradição
author_school_df = pd.read_csv(
    "../notebooks/map-author-school.csv"
)

# Normalização básica
author_school_df["author"] = author_school_df["author"].str.strip()
refs_df["author"] = refs_df["author"].str.strip()

# 2. Left join via author
refs_df = (
    refs_df
    .merge(
        author_school_df,
        on="author",
        how="left"
    )
)

# 3. Se tradition não foi encontrada, marcar como 'non-classified'
refs_df["tradition"] = refs_df["tradition"].fillna("non-classified")

refs_df.head()


,paper_id,raw,co_cited_count,section_id,paragraph_id,sentence_id,sentence_seq_number,reference_seq_number,author,page,year,title,filename,sentence_count,reference_count,source title,similarity,tradition
0,1,Streissler (1972),0,_gzuY8cP,_NwgSU35,_TMX2wdg,4,1,Streissler,NaN,1972.0,Unknown Title,10.1002.9780470999059.ch17.pdf.grobid.tei.xml,298,20,Review of Austrian Economics,44.444444,non-classified
1,1,"(Menger, [1981] 1871, p. 49)",1,_NytzfyY,_Z99Bvn5,_aJz9uAb,10,2,Menger,49.0,1981.0,Unknown Title,10.1002.9780470999059.ch17.pdf.grobid.tei.xml,298,20,Review of Austrian Economics,44.444444,traditional-austrian
2,1,"(Menger, [1981] 1871, p. 49)",1,_NytzfyY,_Z99Bvn5,_aJz9uAb,10,2,Menger,49.0,1981.0,Unknown Title,10.1002.9780470999059.ch17.pdf.grobid.tei.xml,298,20,Review of Austrian Economics,44.444444,traditional-austrian
3,1,"1871, p. 49)",1,_NytzfyY,_Z99Bvn5,_aJz9uAb,10,3,NaN,49.0,1871.0,Unknown Title,10.1002.9780470999059.ch17.pdf.grobid.tei.xml,298,20,Review of Austrian Economics,44.444444,non-classified
4,1,"(Menger, [1981] 1871, p. 48)",0,_NytzfyY,_Z99Bvn5,_54sgkMV,11,4,Menger,48.0,1981.0,Unknown Title,10.1002.9780470999059.ch17.pdf.grobid.tei.xml,298,20,Review of Austrian Economics,44.444444,traditional-austrian


In [102]:
pd.set_option('display.max_rows', None)


refs_df[['author','tradition']].value_counts(dropna=False).head(100)

author       tradition                  
NaN          non-classified                 38098
Hayek        traditional-austrian           12562
Mises        traditional-austrian            7754
Kirzner      contemporary-austrian           3013
Rothbard     traditional-austrian            2647
Menger       traditional-austrian            2580
Friedman     chicago                         2116
Schumpeter   neoclassical                    1992
Boettke      contemporary-austrian           1629
Buchanan     public-choice                   1485
Smith        classical                       1404
Foss         organizational-theory           1289
Lachmann     traditional-austrian            1230
Keynes       keynesian                       1022
Knight       chicago                          933
Weber        sociology                        909
Shane        non-classified                   823
Williamson   new-institutional-economics      779
Bylund       contemporary-austrian            762
North    

In [103]:
def filter_top_k_authors(refs_df, k):
    """
    Mantém apenas os top-K autores mais frequentes em refs_df.
    
    Popularidade = número de ocorrências do autor no dataframe.
    """

    # 1. Conta frequência por autor
    author_counts = (
        refs_df["author"]
        .value_counts()
    )

    # 2. Seleciona top-K autores
    top_k_authors = set(author_counts.head(k).index)

    # 3. Filtra o dataframe
    filtered_refs_df = refs_df[
        refs_df["author"].isin(top_k_authors)
    ].copy()

    return filtered_refs_df


# Mantém apenas os 100 autores mais citados
#refs_df = filter_top_k_authors(refs_df, k=200)

print(refs_df.shape)



(275255, 18)


In [104]:
refs_df['author'].value_counts().head(10)

author
Hayek         12562
Mises          7754
Kirzner        3013
Rothbard       2647
Menger         2580
Friedman       2116
Schumpeter     1992
Boettke        1629
Buchanan       1485
Smith          1404
Name: count, dtype: int64

In [105]:
import pandas as pd

mises_refs_df = pd.read_csv("../data/processed/mises_refs.csv")

# Seleciona apenas o que interessa do mises_refs_df
mises_parts = (
    mises_refs_df[
        ["sentence_id", "author", "human_action_part_number"]
    ]
    .dropna(subset=["sentence_id", "human_action_part_number"])
    .copy()
)

# Faz LEFT JOIN em refs_df
refs_df = refs_df.merge(
    mises_parts,
    on=["sentence_id", "author"],
    how="left"
)


def build_author_node(row):
    if (
        row["author"] == "Mises"
        and pd.notna(row["human_action_part_number"])
    ):
        return f"Mises_{(row['human_action_part_number'])}"
    return row["author"]


refs_df["author"] = refs_df.apply(build_author_node, axis=1)


In [106]:
refs_df[['author', 'sentence_id', 'paragraph_id', 'section_id', 'paper_id']].head(5)

,author,sentence_id,paragraph_id,section_id,paper_id
0,Streissler,_TMX2wdg,_NwgSU35,_gzuY8cP,1
1,Menger,_aJz9uAb,_Z99Bvn5,_NytzfyY,1
2,Menger,_aJz9uAb,_Z99Bvn5,_NytzfyY,1
3,NaN,_aJz9uAb,_Z99Bvn5,_NytzfyY,1
4,Menger,_54sgkMV,_Z99Bvn5,_NytzfyY,1


In [107]:
refs_df[['author', 'sentence_id', 'paragraph_id', 'section_id', 'paper_id']] \
    .isna() \
    .mean() \
    .sort_values(ascending=False).head(1)


author    0.138267
dtype: float64

In [108]:
import pandas as pd

def percentage_units_with_mises(refs_df):
    granularities = {
        "sentence": "sentence_id",
        "paragraph": "paragraph_id",
        "section": "section_id",
        "paper": "paper_id",
    }

    results = []

    for level_name, level_col in granularities.items():

        # total de unidades naquele nível
        total_units = (
            refs_df[level_col]
            .dropna()
            .nunique()
        )

        if total_units == 0:
            continue

        # unidades que contêm Mises
        mises_units = (
            refs_df.loc[refs_df["author"] == "Mises", level_col]
            .dropna()
            .nunique()
        )

        percent = 100 * mises_units / total_units

        results.append({
            "granularity": level_name,
            "total_units": total_units,
            "units_with_mises": mises_units,
            "percent_with_mises": percent
        })

    return pd.DataFrame(results)

mises_coverage_df = percentage_units_with_mises(refs_df)
mises_coverage_df


,granularity,total_units,units_with_mises,percent_with_mises
0,sentence,190971,15,0.007855
1,paragraph,97610,15,0.015367
2,section,26404,14,0.053022
3,paper,2643,10,0.378358


In [109]:
import random

# --------------------------------------------------
# papers que contêm Mises
# --------------------------------------------------
papers_with_mises = set(
    refs_df.loc[refs_df["author"] == "Mises", "paper_id"]
    .dropna()
    .unique()
)

# --------------------------------------------------
# todos os papers com título
# --------------------------------------------------
papers_df = (
    refs_df[["paper_id", "title", "filename"]]
    .dropna(subset=["paper_id"])
    .drop_duplicates(subset=["paper_id"])
)

# --------------------------------------------------
# papers sem Mises
# --------------------------------------------------
papers_without_mises_df = papers_df[
    ~papers_df["paper_id"].isin(papers_with_mises)
]

# --------------------------------------------------
# sorteia um paper
# --------------------------------------------------
random_row = papers_without_mises_df.sample(n=1, random_state=None)

random_row


,paper_id,title,filename
43062,247,From contract to mental model: Constitutional ...,10.1007.s11138-009-0086-4.pdf.grobid.tei.xml


In [110]:
def pair_metrics_by_granularity(
    refs_df,
    author_a,
    author_b,
    granularities=("sentence_id", "paragraph_id", "section_id", "paper_id")
):
    results = []

    for g in granularities:
        df = refs_df[[g, "author"]].dropna()

        units = (
            df
            .groupby(g)["author"]
            .agg(set)
        )

        num_units = len(units)

        if num_units == 0:
            results.append({
                "granularity": g,
                "num_units": 0,
                "support_abs": 0,
                "support": 0.0,
                "confidence": 0.0,
                "lift": 0.0
            })
            continue

        has_a = units.apply(lambda s: author_a in s)
        has_b = units.apply(lambda s: author_b in s)

        count_a = has_a.sum()
        count_b = has_b.sum()
        count_ab = (has_a & has_b).sum()

        if count_a == 0 or count_b == 0:
            confidence = 0.0
            lift = 0.0
            support = 0.0
        else:
            support = count_ab / num_units
            confidence = count_ab / count_a
            lift = confidence / (count_b / num_units)

        results.append({
            "granularity": g,
            "num_units": num_units,
            "support_abs": int(count_ab),
            "support": support,
            "confidence": confidence,
            "lift": lift
        })

    return pd.DataFrame(results)


In [111]:
metrics = pair_metrics_by_granularity(
    refs_df,
    author_a="Lange",
    author_b="Mises"
)

metrics

,granularity,num_units,support_abs,support,confidence,lift
0,sentence_id,160560,0,0.000000,0.000000,0.000000
1,paragraph_id,83708,0,0.000000,0.000000,0.000000
2,section_id,22464,0,0.000000,0.000000,0.000000
3,paper_id,2409,1,0.000415,0.010753,2.590323


In [112]:
def precompute_target_stats(refs_df, target_author="Mises"):
    granularities = ["sentence_id", "paragraph_id", "section_id", "paper_id"]

    stats = {}

    for g in granularities:
        units = (
            refs_df
            .dropna(subset=[g, "author"])
            .groupby(g)["author"]
            .apply(set)
        )

        num_units = len(units)
        units_with_target = units.apply(lambda s: target_author in s)
        support_target = units_with_target.mean()

        stats[g] = {
            "num_units": num_units,
            "support_target": support_target,
            "units": units  # ⚠️ cache estrutural
        }

    return stats


In [113]:
def pair_metrics_from_cache(
    author,
    target_author,
    target_stats
):
    rows = []

    for g, data in target_stats.items():
        units = data["units"]

        both = units.apply(
            lambda s: author in s and target_author in s
        )

        support_abs = both.sum()
        support = support_abs / data["num_units"]

        support_author = units.apply(lambda s: author in s).mean()

        confidence = (
            support / support_author
            if support_author > 0
            else 0.0
        )

        lift = (
            confidence / data["support_target"]
            if data["support_target"] > 0
            else 0.0
        )

        rows.append({
            "granularity": g,
            "num_units": data["num_units"],
            "support_abs": support_abs,
            "support": support,
            "confidence": confidence,
            "lift": lift,
        })

    return pd.DataFrame(rows)


In [114]:
import pandas as pd


def build_authors_df(
    refs_df,
    top_k_authors,
    target_author="Mises"
):
    target_stats = precompute_target_stats(
        refs_df,
        target_author
    )

    dfs = []

    for author in top_k_authors:
        df = pair_metrics_from_cache(
            author,
            target_author,
            target_stats
        )

        if df.empty:
            continue

        df = df.assign(
            author=author,
            with_author=target_author
        )

        dfs.append(df)

    if not dfs:
        return pd.DataFrame()

    authors_df = pd.concat(dfs, ignore_index=True)[
        ["author", "with_author", "granularity",
         "num_units", "support_abs", "support",
         "confidence", "lift"]
    ]

    # 🔹 Mapa único author → tradition (vem do refs_df)
    author_tradition = (
        refs_df[["author", "tradition"]]
        .dropna(subset=["author"])
        .drop_duplicates(subset=["author"])
    )

    # 🔹 Left join
    authors_df = authors_df.merge(
        author_tradition,
        on="author",
        how="left"
    )

    return authors_df


In [115]:
import pandas as pd


def get_top_k_authors(refs_df, k=20, exclude=("Mises",)):
    vc = refs_df["author"].value_counts()
    vc = vc.drop(labels=exclude, errors="ignore")
    return vc.head(k).index.tolist()


In [116]:
pd.set_option("display.max_rows", None)

K = 1000

target_author = "Knight"

top_k_authors = get_top_k_authors(
    refs_df,
    k=K,
    exclude={target_author}
)

print("Top K authors generated.")

authors_df = build_authors_df(
    refs_df,
    top_k_authors,
    target_author=target_author
)

authors_df.to_csv("../data/processed/author_lifts.csv", encoding="utf-8-sig")


Top K authors generated.


In [117]:
authors_df.head()

,author,with_author,granularity,num_units,support_abs,support,confidence,lift,tradition
0,Hayek,Knight,sentence_id,160560,39,0.000243,0.006324,1.090632,traditional-austrian
1,Hayek,Knight,paragraph_id,83708,76,0.000908,0.015657,1.648594,traditional-austrian
2,Hayek,Knight,section_id,22464,144,0.006410,0.048567,1.756844,traditional-austrian
3,Hayek,Knight,paper_id,2409,251,0.104193,0.193822,1.206507,traditional-austrian
4,Mises_WHOLE,Knight,sentence_id,160560,97,0.000604,0.031627,5.454383,traditional-austrian


In [118]:
import pandas as pd

# --------------------------------------------------
# Configuração
# --------------------------------------------------

GRANULARITY_ORDER = [
    "sentence_id",
    "paragraph_id",
    "section_id",
    "paper_id"
]

def lift_to_sign(lift: float) -> str:
    if lift > 1:
        return "+"
    elif lift < 1:
        return "-"
    else:
        return "0"   # opcional


# --------------------------------------------------
# Função principal
# --------------------------------------------------

def build_pattern_table_from_authors_df(authors_df: pd.DataFrame) -> pd.DataFrame:
    """
    A partir de authors_df (uma linha por author × with_author × granularidade),
    computa:
      - pattern de sinais de lift ao longo da granularidade
      - count ponderado por support_abs

    Retorna tabela ordenada por count (desc):
      author | with_author | tradition | pattern | count
    """

    df = authors_df.copy()

    # Sinal do lift
    df["sign"] = df["lift"].apply(lift_to_sign)

    # Garantir ordem das granularidades
    df["granularity"] = pd.Categorical(
        df["granularity"],
        categories=GRANULARITY_ORDER,
        ordered=True
    )

    # Pivot dos sinais
    pivot_sign = (
        df
        .pivot_table(
            index=["author", "with_author", "tradition"],
            columns="granularity",
            values="sign",
            aggfunc="first"
        )
        .add_suffix("_sign")
    )

    # Pivot do support_abs
    pivot_support = (
        df
        .pivot_table(
            index=["author", "with_author", "tradition"],
            columns="granularity",
            values="support_abs",
            aggfunc="sum"
        )
        .add_suffix("_support")
    )

    # Junta tudo
    pivot = (
        pivot_sign
        .join(pivot_support)
        .reset_index()
    )

    # Construir pattern usando as colunas *_sign
    sign_cols = [f"{g}_sign" for g in GRANULARITY_ORDER]
    pivot["pattern"] = pivot[sign_cols].apply(
        lambda row: "".join(row.values.astype(str)),
        axis=1
    )

    # Definir count como soma do support_abs
    support_cols = [f"{g}_support" for g in GRANULARITY_ORDER]
    pivot["count"] = pivot[support_cols].sum(axis=1)

    # Tabela final
    table = (
        pivot[["author", "with_author", "tradition", "pattern", "count"]]
        .sort_values("count", ascending=False)
        .reset_index(drop=True)
    )

    return table


# --------------------------------------------------
# Uso
# --------------------------------------------------

pattern_table = build_pattern_table_from_authors_df(authors_df)

pattern_table.to_csv("../data/processed/pattern_table.csv", encoding="utf-8-sig")

pattern_table.head(20)


C:\Users\pedro\AppData\Local\Temp\ipykernel_14648\105602256.py:53: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  .pivot_table(
C:\Users\pedro\AppData\Local\Temp\ipykernel_14648\105602256.py:65: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  .pivot_table(


granularity,author,with_author,tradition,pattern,count
0,Mises_WHOLE,Knight,traditional-austrian,++++,677
1,Kirzner,Knight,contemporary-austrian,++++,628
2,Schumpeter,Knight,neoclassical,++++,564
3,Hayek,Knight,traditional-austrian,++++,510
4,Foss,Knight,organizational-theory,++++,446
5,Klein,Knight,contemporary-austrian,++++,245
6,Smith,Knight,classical,++++,240
7,Shane,Knight,non-classified,++++,233
8,Casson,Knight,non-classified,++++,214
9,Baumol,Knight,non-classified,++++,210


In [119]:
import pandas as pd

granularity_order = [
    "sentence_id",
    "paragraph_id",
    "section_id",
    "paper_id"
]

df = authors_df.copy()

# 1. Sinal do lift
if "lift_relation" not in df.columns:
    df["lift_relation"] = df["lift"].apply(lambda x: "+" if x >= 1 else "-")

# 2. Ordem das granularidades
df["granularity"] = pd.Categorical(
    df["granularity"],
    categories=granularity_order,
    ordered=True
)

# 3. Uma linha por author × tradition
author_patterns = (
    df
    .pivot_table(
        index=["author", "tradition"],
        columns="granularity",
        values="lift_relation",
        aggfunc="first"
    )
    .reindex(columns=granularity_order)
)

# 4. Apenas patterns completos
author_patterns = author_patterns.dropna()

# 5. Pattern multigranular
author_patterns["pattern"] = author_patterns.apply(
    lambda row: " ".join(row.values),
    axis=1
)

author_patterns = author_patterns.reset_index()

# 🔹 Total de autores por tradition
tradition_totals = (
    author_patterns
    .groupby("tradition")["author"]
    .nunique()
    .rename("total_authors")
    .reset_index()
)

# 6. Contagem por tradition × pattern
pattern_by_tradition = (
    author_patterns
    .groupby(["tradition", "pattern"])
    .size()
    .rename("count")
    .reset_index()
)

# 7. Junta total e calcula proporção correta
pattern_by_tradition = pattern_by_tradition.merge(
    tradition_totals,
    on="tradition",
    how="left"
)

pattern_by_tradition["proportion"] = (
    pattern_by_tradition["count"] /
    pattern_by_tradition["total_authors"]
)

# 8. Ordenação para inspeção
pattern_by_tradition.sort_values(
    ["tradition", "proportion", "pattern"],
    ascending=[True, False, True]
).head(100)


C:\Users\pedro\AppData\Local\Temp\ipykernel_14648\1594686634.py:26: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  .pivot_table(


,tradition,pattern,count,total_authors,proportion
0,chicago,+ + + +,2,6,0.333333
1,chicago,- + + +,2,6,0.333333
2,chicago,- - - +,2,6,0.333333
3,classical,+ + + +,5,10,0.500000
5,classical,- - + +,2,10,0.200000
4,classical,+ - + +,1,10,0.100000
6,classical,- - - +,1,10,0.100000
7,classical,- - - -,1,10,0.100000
14,contemporary-austrian,- - - -,7,21,0.333333
8,contemporary-austrian,+ + + +,3,21,0.142857
